<a href="https://colab.research.google.com/github/ObjectMatrix/Automotive-Physical-Damage-Claimss/blob/main/automative.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import nexessary libs

In [ ]:
!pip install tensorflow

In [2]:
import tensorflow as tf
# For data augmentation while data preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Pretrained MobileNet model.
from tensorflow.keras.applications import MobileNetV2

# Performing MaxPooling operations
from tensorflow.keras.layers import MaxPooling2D

# For performing dropout operation
from tensorflow.keras.layers import Dropout

# For flattening operation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

# Input() is used to instantiate a Keras tensor.
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

# We use adam optimizer
from tensorflow.keras.optimizers.legacy import Adam

# Preprocesses a tensor or Numpy array encoding a batch of images.
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Converts a PIL Image instance to a Numpy array.
from tensorflow.keras.preprocessing.image import img_to_array

# Loads an image into PIL format.
from tensorflow.keras.preprocessing.image import load_img

# Converts a class vector (integers) to binary class matrix.
from tensorflow.keras.utils import to_categorical

# Binarize labels in a one-vs-all fashion.
from sklearn.preprocessing import LabelBinarizer

# Performing train-test split
from sklearn.model_selection import train_test_split

# For printing the metrics
from sklearn.metrics import classification_report

# For using strings in file structure path format
from imutils import paths

# For plotting losse functions
import matplotlib.pyplot as plt

# For performing mathematical computations
import numpy as np

# For file-related operations
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# Path to directory containing data
DataDir = "/content/drive/MyDrive/input/car-damage-detection/data1a"
# Path to training directory
train_dir = os.path.join(DataDir, 'training/')

# Path to validation directory
val_dir = os.path.join(DataDir, 'validation/')

# Path for damaged training images
train_damage = os.path.join(train_dir, '00-damage')
# Path for training images not damaged
train_not_damage = os.path.join(val_dir, '01-whole')

# Number of damaged training images
num_train_damage = len(os.listdir(train_damage))

# Number of training images not damaged
num_train_not_damage = len(os.listdir(train_not_damage))

# Path for damaged validation
val_damage = os.path.join(val_dir, '00-damage')

# Path for validation images not damaged
val_not_damage = os.path.join(val_dir, '01-whole')

# Number of damaged validation images
num_val_damage = len(os.listdir(val_damage))

# Number of validation images not damaged
num_val_not_damage = len(os.listdir((val_not_damage)))

# Number of training images
num_train = num_train_damage + num_train_not_damage

# Number of validation images
num_val = num_val_damage + num_val_not_damage


# Total images
total_images = num_val + num_train
print("Total training images",num_train)
print("Total training images (Damaged)", num_train_damage)
print("Total training images (Damaged)", num_train_not_damage)
print()

print("Total validation images", num_val)
print("Total training images (Damaged)", num_val_damage)
print("Total training images (Damaged)", num_val_not_damage)
print()

print("Total Number of Images: ",total_images)

# Plotting a sample image
plt.grid('')
image = plt.imread('/content/drive/MyDrive/input/car-damage-detection/data1a/training/01-whole/0195.jpg')
plt.imshow(image)
plt.show()

In [9]:
# initializing the hyperparameters
initial_lr = 0.001
epochs = 100
batch_size = 64

# Classes which are detected
classes = ["00-damage", "01-whole"]

In [10]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")

# It stores the data or feature set
data = []

# It stores the corrosponding labels
labels = []

for class_ in classes:
    path = os.path.join(train_dir, class_)
    for image in os.listdir(path):
    	image_path = os.path.join(path, image)
    	image_ = load_img(image_path, target_size=(224, 224))
    	image_ = img_to_array(image_)
    	image_ = preprocess_input(image_)

    	data.append(image_)
    	labels.append(class_)


for class_ in classes:
    path = os.path.join(val_dir, class_)
    for image in os.listdir(path):
    	image_path = os.path.join(path, image)
    	image_ = load_img(image_path, target_size=(224, 224))
    	image_ = img_to_array(image_)
    	image_ = preprocess_input(image_)

    	data.append(image_)
    	labels.append(class_)

# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

data = np.array(data, dtype="float32")
labels = np.array(labels)



(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.20, stratify=labels, random_state=42)
# construct the training image generator for data augmentation
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

[INFO] loading images...


In [12]:
# loading the MobileNetV2 network, ensuring the topmost fully-connected
# layer sets are left off
model_base = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))
# Constructing the top architecture of our model, which is placed over the
# pretrained model
model_head = model_base.output
# MaxPooling layer

model_head = MaxPooling2D(pool_size=(5, 5))(model_head)
# Flatten layer
model_head = Flatten(name="flatten")(model_head)

# Activation function relu
model_head = Dense(128, activation="relu")(model_head)

# Performing dropout
model_head = Dropout(0.5)(model_head)

# Final output layer consists of softmax layer
model_head = Dense(2, activation="softmax")(model_head)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model_final = Model(inputs=model_base.input, outputs=model_head)


In [ ]:
# looping over all the layers and setting each individual layer trainability
# to false.
for layer in model_base.layers:
	layer.trainable = False

# Setting optimizer to Adam
optim = Adam(lr=initial_lr, decay=initial_lr / epochs)

# Compiling our model
model_final.compile(loss="binary_crossentropy", optimizer=optim,
	metrics=["accuracy"])


# train the head of the network
model_train = model_final.fit(
#     Generates image generator from ImageGeneratorClass for inputing images in batches
	aug.flow(trainX, trainY, batch_size=batch_size),
#     Number of steps to be taken in one epoch over image batches
	steps_per_epoch=len(trainX) // batch_size,
#     Validation data
	validation_data=(testX, testY),

#     Steps for validation data
	validation_steps=len(testX) // batch_size,

#     Number of epochs
	epochs=epochs)

In [ ]:
# Now, we predict on test set.
predict = model_final.predict(testX, batch_size=batch_size)

# for each image in the test set we find the index of the
# label with corresponding largest predicted probability
predict_index = np.argmax(predict, axis=1)

# Displaying classification report
print(classification_report(testY.argmax(axis=1), predict_index,
	target_names=lb.classes_))

model_save_path = "/content/drive/MyDrive/my_car_model/Car_detection.model"
model_final.save(model_save_path, save_format="h5")

# Storing our model for further use.
# model_final.save("Car_detection.model", save_format="h5")

In [30]:
#TEST MODEL HERE
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

#image_path = "/content/drive/MyDrive/input/car-damage-detection/data1a/validation/00-damage/0005.JPEG"
image_path = "/content/drive/MyDrive/input/car-damage-detection/data1a/validation/01-whole/dent.jpg"
# Load the image with the right target size for your model
image = load_img(image_path, target_size=(224, 224))

# Convert the image to a numpy array
image_array = img_to_array(image)

# Add a new axis to make the array match the model's input shape
image_array = np.expand_dims(image_array, axis=0)

# Use the appropriate preprocess input function for your model architecture
image_array = preprocess_input(image_array)

prediction = model_final.predict(image_array)

# Assuming your model's final layer is a softmax, the prediction will be a 2-element array
print(prediction)

# Decode the prediction
predicted_class_index = np.argmax(prediction, axis=1)[0]
classes = ["Damaged", "Not Damaged"]  # Ensure this matches the classes used during training
print(f"Predicted class: {classes[predicted_class_index]}")

1/1 [==============================] - 0s 107ms/step
[[0.9988201  0.00117982]]
Predicted class: Damaged


In [ ]:
# plotting training loss and accuracy
num = epochs

# matplotlib style
plt.style.use("seaborn")
plt.figure()

# Plotting all the required quantities
# Train Accuracy
plt.plot(np.arange(0, num), model_train.history["accuracy"], label="Train Accuracy")

# Validation Accuracy
plt.plot(np.arange(0, num), model_train.history["val_accuracy"], label="Validation Accuracy")

# Train Loss
plt.plot(np.arange(0, num), model_train.history["loss"], label="Train Loss")

# Validation Loss
plt.plot(np.arange(0, num), model_train.history["val_loss"], label="Validation Loss")

# Setting the title
plt.title("Training Loss and Accuracy")

# Setting the label
plt.xlabel("Epochs")
plt.ylabel("Accuracy/Loss")
plt.legend(loc="upper right")

# Saving the Model
plt.savefig("Car_Detection.png")

To enable a model to identify specific types of car damage (e.g., dents, scratches, shattered windows), you would need to:

Collect and Label a More Detailed Dataset: You would need a dataset that includes images of cars with various types of damage, each labeled according to the type of damage it exhibits. The dataset should be diverse and cover all the types of damage you want the model to recognize.

Design a Model for Multi-Class Classification: Modify the existing model or design a new model that can handle multi-class classification. This involves changing the final layer of the model to have as many neurons as there are damage types (classes) and using a softmax activation function. The loss function typically used for multi-class classification is categorical_crossentropy.

Train the Model on the Detailed Dataset: Train the model on the new dataset. This will require splitting the dataset into training, validation, and test sets, performing data augmentation (if necessary), and then training the model using these sets.

Evaluate and Adjust the Model: Evaluate the model's performance using the test set and, if necessary, adjust the model's architecture, hyperparameters, or training procedure to improve its ability to accurately classify different types of damage.

Here is a simplified example of how the final layer of the model might be adjusted for multi-class classification to identify different types of damage:

# Assuming 'model_base' is the base MobileNetV2 model

# Adjust the model head for multi-class classification
model_head = Flatten(name="flatten")(model_base.output)
model_head = Dense(128, activation="relu")(model_head)
model_head = Dropout(0.5)(model_head)

# Adjust the number of neurons in the final layer to match the number of classes
# For example, if there are 5 types of damage, set the final Dense layer to have 5 neurons
num_classes = 5  # Adjust this to match the number of damage types in your dataset
model_head = Dense(num_classes, activation="softmax")(model_head)

# Create the final model
model_final = Model(inputs=model_base.input, outputs=model_head)